In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm import tqdm
from tqdm import tqdm_notebook
import warnings
from sklearn.preprocessing import LabelEncoder,StandardScaler
import gc
import os
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
from sklearn import metrics
from sklearn.metrics import roc_auc_score

from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout
import keras
from keras import backend as K

plt.style.use('seaborn')
sns.set(font_scale=1)
pd.set_option('display.max_columns', 500)
epochs = 3
batch_size = 1024
classes = 1

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')
label = pd.read_csv('train_label.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('submission.csv')

# 数据探索

In [3]:
df_source = train.merge(label,on='ID',how='left')

# 特征工程

In [4]:
test['label'] = -1
train = train.merge(label,on='ID',how='left')
data = pd.concat([train, test])

In [5]:
def get_time_fe(df):
    df['day'] = df.date.apply(lambda x:int(x[8:10]))

    df['hour'] = df.date.apply(lambda x:int(x[11:13]))

    return df

In [6]:
#对时间分箱
def getSeg(x):
    if x >=0 and x<= 6:
        return 1
    elif x>=7 and x<=12:
        return 2
    elif x>=13 and x<=18:
        return 3
    elif x>=19 and x<=23:
        return 4

In [7]:
# hourDF = train_df.groupby(['hour', 'label'])['hour'].count().unstack('label').fillna(0)
# hourDF[[0,1]].plot(kind='bar', stacked=True);

In [8]:
#count统计特征
cross_feature = []
def get_cross_fe(df):
    first_feature = ['B1', 'B2', 'B3']
    second_feature = ['C1','C2','C3','D1','D2']
    for feat_1 in first_feature:
        for feat_2 in second_feature:
            col_name = "cross_" + feat_1 + "_and_" + feat_2
            cross_feature.append(col_name)
            df[col_name] = df[feat_1].astype(str).values + '_' + df[feat_2].astype(str).values
    return df

In [9]:
data = get_time_fe(data)
# data['hour_seg'] = data['hour'].apply(lambda x: getSeg(x))
data = get_cross_fe(data)

In [10]:
#labelencoder()
cate_feature = ['A1','A2','A3','B1','B2','B3','C1','C2','C3','E2','E3','E5','E7','E9','E10','E13','E16','E17','E19','E21','E22']
cross_feature = cross_feature[:15]
cate_features = cate_feature+cross_feature
for item in cate_features:
    data[item] = LabelEncoder().fit_transform(data[item])

In [11]:
def feature_count(data, features=[]):
    new_feature = 'count'
    for i in features:
        new_feature += '_' + i
    try:
        del data[new_feature]
    except:
        pass
    temp = data.groupby(features).size().reset_index().rename(columns={0: new_feature})
    data = data.merge(temp, 'left', on=features)
    return data

for i in cross_feature:
    n = data[i].nunique()
    if n > 5:
        data = feature_count(data, [i])
    else:
        print(i, ':', n)

In [12]:
#ratio：类别偏好的ratio比例特征
label_feature =[ 'A2', 'A3']
data_temp = data[label_feature]
df_feature = pd.DataFrame()
data_temp['cnt'] = 1
print('Begin ratio clcik...')
col_type = label_feature.copy()
n = len(col_type)
for i in range(n):
    col_name = "ratio_click_of_" + col_type[i]
    df_feature[col_name] = (
                    data_temp[col_type[i]].map(data_temp[col_type[i]].value_counts()) / len(data) * 100).astype(int)            
data = pd.concat([data, df_feature], axis=1)
print('The end')

Begin ratio clcik...
The end


In [13]:
train_df = data[data['label'] != -1]
test_df = data[data['label'] == -1]

In [14]:
# sns.stripplot(train_df["label"],train_df["A3"],jitter=True,order=None)
# plt.title("click Vs creative_height");

# 删除不需要的字段

In [15]:
## get train feature
del_feature = ['ID','day','date','label','D2']+cross_feature
features = [i for i in train_df.columns if i not in del_feature]

In [16]:
train_x = train_df[features]
train_y = train_df['label'].values
test = test_df[features]

# 训练

In [17]:
## normalized
scaler = StandardScaler()
train_X = scaler.fit_transform(train_x)
test_X = scaler.transform(test)

In [18]:
## simple mlp model
K.clear_session()
def MLP(dropout_rate=0.25, activation='relu'):
    start_neurons = 512
    model = Sequential()
    model.add(Dense(start_neurons, input_dim=train_X.shape[1], activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    model.add(Dense(start_neurons // 2, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    model.add(Dense(start_neurons // 4, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    model.add(Dense(start_neurons // 8, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate / 2))

    model.add(Dense(classes, activation='sigmoid'))
    return model


In [19]:
def plot_loss_acc(history, fold):
    plt.plot(history.history['loss'][1:])
    plt.plot(history.history['val_loss'][1:])
    plt.title('model loss')
    plt.ylabel('val_loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'Validation'], loc='upper left')
    plt.savefig('../../result/model_loss' + str(fold) + '.png')
    plt.show()

    plt.plot(history.history['acc'][1:])
    plt.plot(history.history['val_acc'][1:])
    plt.title('model Accuracy')
    plt.ylabel('val_acc')
    plt.xlabel('epoch')
    plt.legend(['train', 'Validation'], loc='upper left')
#     plt.savefig('../../result/model_accuracy' + str(fold) + '.png')
    plt.show()

# 五折交叉

In [20]:
classes=1
folds = KFold(n_splits=5, shuffle=True, random_state=2048)
NN_predictions = np.zeros((test_X.shape[0], classes))
oof_preds = np.zeros((train_X.shape[0], classes))

In [21]:
patience = 50   ## How many steps to stop
call_ES = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=1,
                                        mode='auto', baseline=None)

In [22]:
for fold_, (trn_, val_) in enumerate(folds.split(train_x)):
    print("fold {}".format(fold_ + 1))
    x_train, y_train = train_X[trn_], train_y[trn_]
    x_valid, y_valid = train_X[val_], train_y[val_]


    model = MLP(dropout_rate=0.5, activation='relu')
    model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])
    history = model.fit(x_train, y_train,
                        validation_data=[x_valid, y_valid],
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[call_ES, ],
                        shuffle=True,
                        verbose=1)

    # plot_loss_acc(history, fold_ + 1)

    # # Get predicted probabilities for each class
    oof_preds[val_] = model.predict_proba(x_valid, batch_size=batch_size)
    NN_predictions += model.predict_proba(test_X, batch_size=batch_size) / folds.n_splits

fold 1
Train on 48000 samples, validate on 12000 samples
Epoch 1/3
48000/48000 [==============================] - ETA: 4:32 - loss: 0.9080 - acc: 0.540 - ETA: 1:27 - loss: 0.8866 - acc: 0.526 - ETA: 50s - loss: 0.8677 - acc: 0.529 - ETA: 34s - loss: 0.8548 - acc: 0.53 - ETA: 25s - loss: 0.8458 - acc: 0.53 - ETA: 20s - loss: 0.8350 - acc: 0.53 - ETA: 16s - loss: 0.8230 - acc: 0.54 - ETA: 13s - loss: 0.8116 - acc: 0.54 - ETA: 11s - loss: 0.8046 - acc: 0.54 - ETA: 9s - loss: 0.7939 - acc: 0.5535 - ETA: 8s - loss: 0.7891 - acc: 0.557 - ETA: 6s - loss: 0.7820 - acc: 0.560 - ETA: 5s - loss: 0.7764 - acc: 0.564 - ETA: 4s - loss: 0.7706 - acc: 0.567 - ETA: 4s - loss: 0.7648 - acc: 0.571 - ETA: 3s - loss: 0.7598 - acc: 0.575 - ETA: 2s - loss: 0.7550 - acc: 0.579 - ETA: 2s - loss: 0.7497 - acc: 0.583 - ETA: 1s - loss: 0.7455 - acc: 0.585 - ETA: 1s - loss: 0.7405 - acc: 0.589 - ETA: 1s - loss: 0.7356 - acc: 0.592 - ETA: 0s - loss: 0.7314 - acc: 0.596 - ETA: 0s - loss: 0.7276 - acc: 0.598 - 7s 154

Epoch 3/3
48000/48000 [==============================] - ETA: 1s - loss: 0.5221 - acc: 0.810 - ETA: 1s - loss: 0.5314 - acc: 0.801 - ETA: 1s - loss: 0.5313 - acc: 0.800 - ETA: 1s - loss: 0.5313 - acc: 0.803 - ETA: 1s - loss: 0.5297 - acc: 0.806 - ETA: 0s - loss: 0.5305 - acc: 0.804 - ETA: 0s - loss: 0.5335 - acc: 0.801 - ETA: 0s - loss: 0.5317 - acc: 0.802 - ETA: 0s - loss: 0.5305 - acc: 0.803 - ETA: 0s - loss: 0.5286 - acc: 0.804 - ETA: 0s - loss: 0.5261 - acc: 0.805 - ETA: 0s - loss: 0.5261 - acc: 0.806 - ETA: 0s - loss: 0.5254 - acc: 0.806 - ETA: 0s - loss: 0.5261 - acc: 0.806 - ETA: 0s - loss: 0.5248 - acc: 0.806 - ETA: 0s - loss: 0.5243 - acc: 0.806 - ETA: 0s - loss: 0.5222 - acc: 0.807 - ETA: 0s - loss: 0.5216 - acc: 0.807 - ETA: 0s - loss: 0.5210 - acc: 0.807 - ETA: 0s - loss: 0.5204 - acc: 0.808 - ETA: 0s - loss: 0.5204 - acc: 0.808 - ETA: 0s - loss: 0.5187 - acc: 0.809 - ETA: 0s - loss: 0.5181 - acc: 0.809 - 1s 28us/step - loss: 0.5174 - acc: 0.8098 - val_loss: 0.4588 - val_ac

# roc评估

In [23]:
score=roc_auc_score(train_y,oof_preds)
score

0.6248683825016098

In [24]:
sub['label'] = NN_predictions

In [25]:
sub['label'] = sub['label'].apply(lambda x:'%.2f'%x)

In [26]:
label.label.value_counts()

0    49756
1    10244
Name: label, dtype: int64

In [27]:
# sub.label.value_counts()

In [28]:
sub.to_csv('C:/Users/DELL/Desktop/sub/'+str('%.6f'%score)+'keras.csv',encoding='utf-8',index=False)

# feature_importance展示

In [29]:
## plot feature importance
cols = (feature_importance_df[["Feature", "importance"]].groupby("Feature").mean().sort_values(by="importance", ascending=False)[:].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)].sort_values(by='importance',ascending=False)
plt.figure(figsize=(8, 14))
sns.barplot(y="Feature",
            x="importance",
            data=best_features.sort_values(by="importance", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()

NameError: name 'feature_importance_df' is not defined

In [ ]:
# def get_more_fe(df):
#     uniq_list = ['C1','E14','A2','E1','E27']
#     for i in uniq_list:
#         tmp = df.groupby(['hour'])[i].nunique().reset_index(name = i+'_h_uniq')
#         df  = df.merge(tmp, on=['hour'], how='left')
#         tmp1 = df.groupby(['date'])[i].nunique().reset_index(name = i+'_d_uniq')
#         df  = df.merge(tmp1, on=['date'], how='left')
#     dev_list = ['D1','D2']
#     for i in dev_list:
#         tmp = df.groupby(['C1'])[i].nunique().reset_index(name = i+'_C1_de_uniq')
#         df  = df.merge(tmp, on=['C1'], how='left')
#         tmp1 = df.groupby(['C2'])[i].nunique().reset_index(name = i+'_C2_de_uniq')
#         df  = df.merge(tmp1, on=['C2'], how='left')
#         tmp2 = df.groupby(['C3'])[i].nunique().reset_index(name = i+'_C3_de_uniq')
#         df  = df.merge(tmp2, on=['C3'], how='left')
#     return df